In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from google_auth_oauthlib import flow
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import requests
import json
import csv

SERVICE_ACCOUNT_FILE = 'key.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE)

bqclient = bigquery.Client(project='iotube-analytics', credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [217]:
COVALENT_KEY = "ckey_10446544a51944ffa7c6603a9e4"
chain = {
    "Polygon": 137,
    "Ethereum": 1,
    "BSC": 56
}
validator = {
    "Polygon": "0xFBe9A4138AFDF1fA639a8c2818a0C4513fc4CE4B",
    "Ethereum": "0xd8165188ccc135b3a3b2a5d2bc3af9d94753d955",
    "BSC": "0x116404F86e97846110EA08cd52fC2882d4AD3123"
}

In [218]:
def getBridgeDataIn(network):
    '''
    Gets inflow bridge data for a given network
    '''
    dfToIotex = pd.DataFrame(columns = ['Network', 'Date', 'Address', 'Transaction Fee', 'Token Name', 'Token Symbol', 'Value'])
    endpoint = "https://api.covalenthq.com/v1/{}/address/{}/transactions_v2/?page-size={}&key={}".format(chain[network], validator[network], 10000, COVALENT_KEY)
    response = requests.get(endpoint)
    JSON_DATA_RAW = response.json()
    JSON_DATA_RAW = JSON_DATA_RAW['data']

    for item in JSON_DATA_RAW['items']:
        temp = []
        temp.append(network)
        temp.append(item['block_signed_at'].split("T", 1)[0]) # Date
        temp.append(item['from_address']) # Address 
        temp.append(item['gas_price'] * item['gas_spent'] * pow(10, -18)) # Transaction Fee
        for event in item['log_events']:
            if(event['decoded']):
                if(event['decoded']['name'] == "Transfer"):
                    for param in event['decoded']['params']:
                        if(param["name"] == "to"):
                            address = param['value'] 
                        if(param["name"] == "value"):
                            value = param['value']
                    name = event['sender_name']
                    symbol = event['sender_contract_ticker_symbol']
                    decimals = -event['sender_contract_decimals']
                    value = int(value) * pow(10,decimals)
        temp.append(name) # Ticker Name
        temp.append(symbol) # Ticker Symbol
        temp.append(value) # Ticker Value
        dfToIotex.loc[len(dfToIotex)] = temp
                            
    dfToIotex['Date'] = pd.to_datetime(dfToIotex['Date'])
    return dfToIotex

In [225]:
polygon = getBridgeDataIn("Polygon")
ethereum = getBridgeDataIn("Ethereum")
bsc = getBridgeDataIn("BSC")
dfToIotex = polygon.append([ethereum, bsc])

In [247]:
inflow = pd.read_csv('https://storage.googleapis.com/iotube/bridgeInflowRaw')
inflow.head()

,Unnamed: 0,Network,Date,Address,Transaction Fee,Token Name,Token Symbol,Value
0,0,Polygon,2021-07-01,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0
1,1,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
2,2,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
3,3,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,5.0
4,4,Polygon,2021-06-17,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0


In [263]:
def crossChainGrouping(inDf):
    inflowDf = inDf.groupby(["Network", "Date"]).agg({'Value': ['count', 'sum', 'mean', 'median'],
                                                                'Transaction Fee': ['sum', 'mean', 'median']})
    inflowDf['Value']['sum'] = inflowDf['Value']['sum'].round()
    inflowDf['Transaction Fee']['sum'] = inflowDf['Transaction Fee']['sum']

    inflowDf = inflowDf.reset_index()
    inflowDf.columns = inflowDf.columns.droplevel(0)
    inflowDf.columns = ['Network', 'Date', 'Frequency', 'Volume', 'Mean Volume', 'Median Volume', 'Transaction Fee', 'Mean Txn Fee', 'Median Txn Fee']
    
    return inflowDf

def tokenGrouping(inDf):
    '''
    Bridge data grouping by token
    '''
    dfInGroupedToken = inDf.groupby(["Token Symbol", "Date"], as_index=False).agg({'Value': ['count', 'sum', 'mean', 'median']})
    dfInGroupedToken['Value']['sum'] = dfInGroupedToken['Value']['sum'].round()

    dfInGroupedToken = dfInGroupedToken.reset_index()
    dfInGroupedToken.columns = dfInGroupedToken.columns.droplevel(0)
    dfInGroupedToken.columns = ['', 'Token Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']

    dfInGroupedPeriod = inDf.groupby(["Network", "Token Symbol", "Date"], as_index=False).agg({'Value': ['count', 'sum', 'mean', 'median']})
    dfInGroupedPeriod['Value']['sum'] = dfInGroupedPeriod['Value']['sum'].round()

    dfInGroupedPeriod = dfInGroupedPeriod.reset_index()
    dfInGroupedPeriod.columns = dfInGroupedPeriod.columns.droplevel(0)
    dfInGroupedPeriod.columns = ['', 'Network', 'Token Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']
    
    return dfInGroupedToken, dfInGroupedPeriod

In [264]:
dfInGroupedToken, dfInGroupedPeriod = tokenGrouping(inflow)
chained = crossChainGrouping(inflow)

<ipython-input-263-887224c81bbe>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInGroupedToken['Value']['sum'] = dfInGroupedToken['Value']['sum'].round()
<ipython-input-263-887224c81bbe>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInGroupedPeriod['Value']['sum'] = dfInGroupedPeriod['Value']['sum'].round()
<ipython-input-263-887224c81bbe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [17]:
query_string = """
SELECT action_hash, timestamp, sender, "Polygon" as Network
FROM `public-data-finance.crypto_iotex.transaction_logs` 
WHERE recipient = "io12s9f9hv4zsr7umy5hxt6g0k0xr4x6pxdp5w998"
"""
outflowdf = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)

In [18]:
outflowdf

,action_hash,timestamp,sender,Network
0,1c84bd1a1525050ed969e1c64a4faa881d45891e0ca725...,2021-06-30 15:15:20+00:00,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,Polygon
1,dd0a5c93c26cbe4f804c38be3793f1fd5493d12ec450cd...,2021-06-11 10:30:05+00:00,io1vyuupg0arwe5djnh3q6w6kmxqrxgx5drsnrakl,Polygon
2,7fecc3233a6383b21832cdb31198cad4cc3d2046a43d06...,2021-06-11 07:07:10+00:00,io1skczngq77r0kmjjmzl0dwtfvaewc8zzg8tjgcj,Polygon
3,881c5b5ba37acbf5f1060a8d8169da6078edd9193bbfd7...,2021-06-10 01:26:50+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon
4,d18d6de29484b6af960a6f576a1e7009f93e89e1d84213...,2021-06-10 01:10:10+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon
5,bb84fe763559e2ada3f886ae9fb508208c70a827930875...,2021-06-10 01:54:15+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon
6,d78081d18a10d0be95d69f24ef8528a90b7da3011ac857...,2021-06-10 23:40:25+00:00,io16xjhq25fhl5j2lfs8a884j30s7d0dtu9zrz9un,Polygon
7,06c634cfdde3ed058eff1309c2d60435487335778598f3...,2021-06-10 02:01:05+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon
8,51b0d0ebf87a0b3b96757ac702bdc995361fc36b238b3f...,2021-06-10 16:38:50+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon
9,b09a54e9a731f34e31c1d2ab81ff84d5e3ed0c6d413bbc...,2021-06-10 01:02:10+00:00,io1yvvlkfcrzlkewye0dc6uaqc4gnvn5wfqkq0ctn,Polygon


In [13]:
ENDPOINT = "https://analytics.iotexscan.io/query"
query_string = """
query{
  xrc20{
    byAddress(address: "io12s9f9hv4zsr7umy5hxt6g0k0xr4x6pxdp5w998", page: 1, numPerPage: 20){
      exist
      xrc20 {
        contract
        hash
        timestamp
        from
        to
        quantity
      }
      count
    }
  }
}
"""
r = requests.post(ENDPOINT, json={'query': query_string})
df_json = json.loads(r.text)
outflow = pd.DataFrame(columns = ['Network', 'Date', 'Address', 'Transaction Fee', 'Token Name', 'Token Symbol', 'Value'])


In [14]:
for action in df_json['data']['xrc20']['byAddress']['xrc20']:
        to_append = []
        to_append.append("Polygon")
        to_append.append(action['timestamp'])
        to_append.append(action['from'])
        to_append.append(0)
        to_append.append("Unknown")
        to_append.append("UNKNOWN")
        to_append.append(action['quantity'])
        a_series = pd.Series(to_append, index = outflow.columns)
        outflow = outflow.append(a_series, ignore_index=True)
outflow['Date'] = pd.to_datetime(outflow['Date'], unit='s')

In [15]:
outflow

,Network,Date,Address,Transaction Fee,Token Name,Token Symbol,Value
0,Polygon,2021-07-01 12:13:05,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,1000000000000000000
1,Polygon,2021-06-30 15:15:20,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,5000000000000000000
2,Polygon,2021-06-17 17:38:45,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,1000000000000000000
3,Polygon,2021-06-17 17:37:50,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,1000000000000000000
4,Polygon,2021-06-17 17:35:50,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,2000000000000000000
5,Polygon,2021-06-17 17:23:50,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,2000000000000000000
6,Polygon,2021-06-17 17:18:15,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,2000000000000000000
7,Polygon,2021-06-15 16:03:20,io1nr0h8mjxywhn3df57mjpr6nxn79z9vjdkvhsxt,0,Unknown,UNKNOWN,100000000000000000
8,Polygon,2021-06-15 07:11:15,io19msajm9hv4u793jvnwcy23plkwzffywjh257sz,0,Unknown,UNKNOWN,2000000000000000000
9,Polygon,2021-06-14 10:14:00,io1dvfjg5xxnzpydnmq2q0n0n0ham2arytkkvrr2h,0,Unknown,UNKNOWN,500000000000000000
